In [1]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
from tensorflow.keras import optimizers
import tensorflow as tf#Use this to check if the GPU is configured correctly
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2022-06-24 00:21:08.495711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 00:21:08.495782: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11984030142590176835
xla_global_id: -1
]


2022-06-24 00:21:12.846743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-24 00:21:12.853420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-24 00:21:12.853440: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-24 00:21:12.853461: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (maryskal-P6689-MD61019): /proc/driver/nvidia/version does not exist


In [2]:
import os
import numpy as np

In [7]:
input_shape = (512,512,3)
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)

Añado una capa de entrada con tres filtros para que me genere tres canales

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(3,3,padding="same", input_shape=(512,512,1), activation='relu'))
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
#avoid overfitting
model.add(layers.Dropout(0.2, name="dropout_out"))
# Set NUMBER_OF_CLASSES to the number of your final predictions.
model.add(layers.Dense(1, activation="softmax", name="fc_out"))
conv_base.trainable = False

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 3)       30        
                                                                 
 efficientnetb0 (Functional)  (None, 16, 16, 1280)     4049571   
                                                                 
 gap (GlobalMaxPooling2D)    (None, 1280)              0         
                                                                 
 dropout_out (Dropout)       (None, 1280)              0         
                                                                 
 fc_out (Dense)              (None, 1)                 1281      
                                                                 
Total params: 4,050,882
Trainable params: 1,311
Non-trainable params: 4,049,571
_________________________________________________________________


## Dataframe

In [12]:
path = '/home/maryskal/Downloads/archive/vinbigdata'

In [13]:
df = pd.read_csv(os.path.join(path, 'train.csv'))

array(['Aortic enlargement', 'Atelectasis', 'Calcification',
       'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration',
       'Lung Opacity', 'No finding', 'Nodule/Mass', 'Other lesion',
       'Pleural effusion', 'Pleural thickening', 'Pneumothorax',
       'Pulmonary fibrosis'], dtype=object)

In [14]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072


In [15]:
for i, finding in enumerate(np.unique(df.class_name)):
    print(f'class name: {finding}')
    print(f'ID: {df.class_id[i]}')
    print(sum(df.class_name == finding))

class name: Aortic enlargement
ID: 14
7162
class name: Atelectasis
ID: 14
279
class name: Calcification
ID: 3
960
class name: Cardiomegaly
ID: 0
5427
class name: Consolidation
ID: 14
556
class name: ILD
ID: 11
1000
class name: Infiltration
ID: 5
1247
class name: Lung Opacity
ID: 8
2483
class name: No finding
ID: 0
31818
class name: Nodule/Mass
ID: 13
2580
class name: Other lesion
ID: 7
2203
class name: Pleural effusion
ID: 13
2476
class name: Pleural thickening
ID: 14
4842
class name: Pneumothorax
ID: 14
226
class name: Pulmonary fibrosis
ID: 0
4655


Creo la clase neumonía

In [16]:
values = ['Consolidation']
df['Neumonia'] = 0
index = [i for i, clase in enumerate(df.class_name) if clase in values]
df['Neumonia'][index] = 1

/tmp/ipykernel_52482/228366499.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Neumonia'][index] = 1


Divido el dataset en dos y reduzco el de no neumonía

In [17]:
df_neumo = df[df.Neumonia == 1].reset_index(drop=True)
df_no_neumo = df[df.Neumonia == 0].reset_index(drop=True)

Para reducir elijo un numero de valores aleatorios que serán los índices

In [20]:
selection = np.random.randint(0, len(df_no_neumo), len(df_neumo))
df_no_neumo = df_no_neumo.iloc[selection,:]

(556, 11)

Genero el nuevo dataframe juntando

In [22]:
df = pd.concat([df_neumo, df_no_neumo]).reset_index(drop=True)

Reordeno aleatoriamente

In [23]:
df = df.sample(frac=1)

## Imagenes

In [24]:
import SimpleITK as sitk

In [25]:
def downsample(img):
    array = sitk.GetArrayFromImage(img)
    ds_array = array/255
    r = np.stack((ds_array,)*1, axis = -1)
    return r

In [26]:
def charge_img(name):
    img = sitk.ReadImage(os.path.join(path, 'train', name + '.png'))
    img = downsample(img)
    return(img)

In [28]:
tensor = np.zeros((1112, 512, 512, 1))
for i in range(1112):
    tensor[i,...] = charge_img(df.image_id[i])

## X e Y

In [27]:
Y = np.array(df['Neumonia'])
X = tensor

In [29]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, 
                                                                    random_state=42, shuffle=True, stratify=Y )

## Compilación y entrenamiento

In [25]:
NUMBER_OF_TRAINING_IMAGES = len(df)
batch_size = 100

In [11]:
opt = optimizers.RMSprop(lr=0.001)
loss = 'categorical_crossentropy'
model.compile(optimizer = opt , loss = loss , metrics = ['accuracy'])

/home/maryskal/anaconda3/envs/acil_init/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [32]:
history = model.fit(X_train,Y_train, batch_size = 50, epochs = 10 , 
                    validation_data = (X_test, Y_test))

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'int'>"})